In [1]:
import os
import pandas as pd
import numpy as np 
import s3fs
import configparser

config = configparser.ConfigParser()
# Este archivo se creará cuando se instale el AWS command line interface.
# Seguir estas instrucciones: https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html
# ES OBLIGATORIO instalar este componente antes de continuar esta ejecución.
config.read(os.getenv("HOME") + '/.aws/credentials')

aws_credentials = {'key': config["cdas-itam"]["aws_access_key_id"],'secret': config["cdas-itam"]["aws_secret_access_key"]}

df = pd.read_csv(f"s3://captain-planet-denuncias-paot/raw/rptDatosAbiertos.csv", storage_options=aws_credentials, encoding="latin-1")

df.head()



,id denuncia,Estatus,Tipo de denuncia,Expediente,Materia,Fecha de recepción,Fecha de ratificación,Fecha de turno,Fecha de admisión/radicación,Fecha de conclusión,...,Alcaldía,Entre calle 1,Entre calle 2,Referencias,Código postal,Régimen propiedad,"Actos, Hechos y Omisiones",Área responsable,Coord. X,Coord. Y
0,1,Concluida,Denuncia ciudadana,PAOT-2002/CAJRD-0001-SOT-001,Áreas Verdes (en Suelo Urbano),30/01/2002,30/01/2002,06/02/2002,06/02/2002,14/03/2002,...,Iztacalco,ORIENTE 217,ORIENTE 221,NaN,8500.0,Pública,Derribo de árboles en área verde de la Colonia...,Subprocuraduría del Ordenamiento Territorial,2144406.000,491217.0000
1,2,Concluida,Denuncia ciudadana,PAOT-2002/CAJRD-0002-SPA-001,Suelo de Conservación,10/04/2002,10/04/2002,12/04/2002,12/04/2002,10/06/2002,...,Tláhuac,NaN,NaN,NaN,13500.0,Social,Contaminación de suelos agrícolas con material...,"Subprocuraduría Ambiental, de Protección y Bie...",2128631.137,500897.8680
2,3,Concluida,Denuncia ciudadana,PAOT-2002/CAJRD-0003-SPA-002,Aire,29/04/2002,29/04/2002,30/04/2002,06/05/2002,24/06/2002,...,Cuauhtémoc,FRANCISCO PIMENTEL,ALFONSO HERRERA,NaN,6470.0,Pública,Contaminación del aire por servicios de hojala...,"Subprocuraduría Ambiental, de Protección y Bie...",2149346.794,482638.9534
3,4,No admitida,Denuncia ciudadana,PAOT-2002/CAJRD-0004-SPA-004,Maltrato Animal,30/04/2002,30/04/2002,02/05/2002,02/05/2002,02/05/2002,...,Álvaro Obregón,NaN,NaN,NaN,1450.0,Privada,Daño en propiedad ajena y amenazas por Pit-bul...,"Subprocuraduría Ambiental, de Protección y Bie...",2141901.002,478549.2810
4,7,Concluida,Denuncia ciudadana,PAOT-2002/CAJRD-0006-SPA-003,Uso de Suelo Urbano,02/05/2002,03/05/2002,07/05/2002,07/05/2002,20/06/2002,...,Álvaro Obregón,NaN,NaN,NaN,1001.0,Pública,Omisión de leyes y normas ambientales con el p...,"Subprocuraduría Ambiental, de Protección y Bie...",2143425.900,479864.3900


Se lee el csv y se guarda como DataFrame

In [2]:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('ó', 'o')
df.columns = df.columns.str.replace('í', 'i')
df.columns = df.columns.str.replace('é', 'e')
df.columns = df.columns.str.replace('á', 'a')

df.columns

Index(['id_denuncia', 'estatus', 'tipo_de_denuncia', 'expediente', 'materia',
       'fecha_de_recepcion', 'fecha_de_ratificacion', 'fecha_de_turno',
       'fecha_de_admision/radicacion', 'fecha_de_conclusion',
       'medio_de_recepcion', 'domicilio', 'colonia', 'alcaldia',
       'entre_calle_1', 'entre_calle_2', 'referencias', 'codigo_postal',
       'regimen_propiedad', 'actos,_hechos_y_omisiones', 'area_responsable',
       'coord._x', 'coord._y'],
      dtype='object')

Se estandariza el nombre de las columnas


In [3]:


pd.to_datetime(df['fecha_de_recepcion'], dayfirst = True, errors='coerce')
pd.to_datetime(df['fecha_de_ratificacion'], dayfirst = True, errors='coerce')
pd.to_datetime(df['fecha_de_turno'], dayfirst = True, errors='coerce')
pd.to_datetime(df['fecha_de_admision/radicacion'], dayfirst = True, errors='coerce')
pd.to_datetime(df['fecha_de_conclusion'], dayfirst = True, errors='coerce')



0       2002-03-14
1       2002-06-10
2       2002-06-24
3       2002-05-02
4       2002-06-20
           ...    
57711          NaT
57712          NaT
57713          NaT
57714          NaT
57715          NaT
Name: fecha_de_conclusion, Length: 57716, dtype: datetime64[ns]

Se limpian y estandarizan las fechas


In [4]:
df['materia'].unique() # materia
df['colonia'].unique() # colonia
df['alcaldia'].unique() #alcaldia
df['medio_de_recepcion'].unique() # medio de recepcion
df['tipo_de_denuncia'].unique() # tipo de denuncia
df['area_responsable'].unique() # area responsable

array(['Subprocuraduría del Ordenamiento Territorial',
       'Subprocuraduría Ambiental, de Protección y Bienestar a los Animales',
       nan], dtype=object)

Se revisa que campos categóricos realmente sean campos categóricos

In [5]:
df['materia'] = df['materia'].astype('category')
df['colonia'] = df['colonia'].astype('category')
df['alcaldia'] = df['alcaldia'].astype('category')
df['regimen_propiedad'] = df['regimen_propiedad'].astype('category')
df['medio_de_recepcion'] = df['medio_de_recepcion'].astype('category')
df['tipo_de_denuncia'] = df['tipo_de_denuncia'].astype('category')
df['area_responsable'] = df['area_responsable'].astype('category')

In [6]:
del df['entre_calle_1']
del df['entre_calle_2']
del df['referencias']

Se borran entre_calle_1, entre_calle_2 y referencias

In [7]:
df['denuncia_padre'] = np.where(df['expediente'].str.contains('BIS'), df['expediente'].shift(1), None)

df['estatus'].unique()


array(['Concluida', 'No admitida', 'No presentada', 'En investigación',
       'En proceso de admisión', 'En proceso de turno'], dtype=object)

Se verifica que denuncias tienen 'BIS' en su expediente

In [8]:
df['coord._x'] = df['coord._x'].astype(float)
df['coord._y'] = df['coord._y'].astype(float)

Se convierten coordenadas a float

In [9]:
df.to_csv(r's3://captain-planet-denuncias-paot/clean/clean_denuncias_paot.csv', storage_options=aws_credentials)

Se exporta en csv para seguir con el análisis